In [3]:
! pip install pyspark
! pip install pandas

In [4]:
from pyspark.sql import SparkSession

packages = [
    "com.amazonaws:aws-java-sdk-s3:1.12.95",
    "org.apache.hadoop:hadoop-aws:3.2.0",
    "org.apache.spark:spark-avro_2.12:2.4.4",
    "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1",
    "org.postgresql:postgresql:42.2.18"

]

spark = SparkSession.builder.appName("dw-dim_date")\
    .master('spark://172.1.0.2:7077')\
    .config("spark.jars.packages", ",".join(packages))\
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")\
    .config("spark.hadoop.fs.s3a.multipart.size", 104857600)\
    .getOrCreate()

spark

:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.amazonaws#aws-java-sdk-s3 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fd6434b0-8870-4d46-8d12-e789fce1450e;1.0
	confs: [default]
	found com.amazonaws#aws-java-sdk-s3;1.12.95 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.95 in central
	found com.amazonaws#aws-java-sdk-core;1.12.95 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found com.fasterxml.jackson.core#jackson-databind;

In [5]:
from datetime import datetime
from datetime import timedelta
import pandas as pd

festival_dict = {'2022-01-01': '2022新曆新年',
                 '2022-02-01': '農曆年初一',
                 '2022-02-02': '農曆年初二',
                 '2022-02-03': '農曆年初三',
                 '2022-02-14': '情人節',
                 '2022-02-15': '元宵節',
                 '2022-03-14': '白色情人節',
                 '2022-04-18': '復活節',
                 '2022-05-08': '母親節',
                 '2022-06-03': '端午節',
                 '2022-06-19': '父親節',
                 '2022-08-04': '七夕節',
                 '2022-09-10': '中秋節',
                 '2022-10-31': '萬聖節',
                 '2022-11-24': '感恩節',
                 '2022-12-25': '聖誕節',
                 '2023-01-01': '2023新曆新年',
                 '2023-01-22': '農曆年初一',
                 '2023-01-23': '農曆年初二',
                 '2023-01-24': '農曆年初三',
                 '2023-02-05': '元宵節',
                 '2023-02-14': '情人節','2023-03-14': '白色情人節',
                 '2023-04-10': '復活節',
                 '2023-05-14': '母親節',
                 '2023-06-19': '父親節',
                 '2023-06-22': '端午節',
                 '2023-08-22': '七夕節',
                 '2023-09-29': '中秋節',
                 '2023-10-31': '萬聖節',
                 '2023-11-23': '感恩節',
                 '2023-12-25': '聖誕節'}
public_holiday_list = ['2022-01-01',
                       '2022-01-01',
                       '2022-02-02',
                       '2022-03-03',
                       '2022-05-05',
                       '2022-15-15',
                       '2022-16-16',
                       '2022-18-18',
                       '2022-02-02',
                       '2022-09-09',
                       '2022-03-03',
                       '2022-01-01',
                       '2022-12-12',
                       '2022-01-01',
                       '2022-04-04',
                       '2022-26-26',
                       '2022-27-27']
good_days_list = ['2022-01-05',
                  '2022-01-16',
                  '2022-01-19',
                  '2022-01-22',
                  '2022-01-28',
                  '2022-01-31',
                  '2022-02-04',
                  '2022-02-11',
                  '2022-02-19',
                  '2022-02-23',
                  '2022-03-04',
                  '2022-03-07',
                  '2022-03-19',
                  '2022-03-24',
                  '2022-03-30',
                  '2022-04-03',
                  '2022-04-09',
                  '2022-04-19',
                  '2022-04-23',
                  '2022-05-05',
                  '2022-05-08',
                  '2022-05-11',
                  '2022-05-15',
                  '2022-05-20',
                  '2022-05-26',
                  '2022-06-01',
                  '2022-06-04',
                  '2022-06-12',
                  '2022-06-14',
                  '2022-06-20',
                  '2022-06-23',
                  '2022-07-05',
                  '2022-07-07',
                  '2022-07-12',
                  '2022-07-15',
                  '2022-07-18',
                  '2022-07-20',
                  '2022-07-25',
                  '2022-07-31',
                  '2022-08-06',
                  '2022-08-09',
                  '2022-08-11',
                  '2022-08-18',
                  '2022-08-21',
                  '2022-08-31',
                  '2022-09-09',
                  '2022-09-14',
                  '2022-09-21',
                  '2022-09-26',
                  '2022-09-30',
                  '2022-10-06',
                  '2022-10-10',
                  '2022-10-17',
                  '2022-10-24',
                  '2022-11-01',
                  '2022-11-03',
                  '2022-11-07',
                  '2022-11-19',
                  '2022-11-25',
                  '2022-12-03',
                  '2022-12-14',
                  '2022-12-26', ]
wedding_expo_list = ['2022-02-11',
                     '2022-02-12',
                     '2022-02-13',
                     '2022-06-03',
                     '2022-06-04',
                     '2022-06-05',
                     '2022-08-26',
                     '2022-08-27',
                     '2022-08-28',
                     '2022-12-09',
                     '2022-12-10',
                     '2022-12-11', ]


dim_date_list = []
new_year = datetime(2022, 1, 1, 0, 0, 0, 0)
for num in range(365*2):
    date = (new_year + timedelta(days=num))
    if date.strftime("%Y-%m-%d") in festival_dict:
        festival = festival_dict[date.strftime("%Y-%m-%d")]
    date_row = {
        "full_date": date,
        "year": int(date.strftime('%Y')),
        "month": int(date.strftime('%m')),
        "day": int(date.strftime('%d')),
        "quarter": int((date.month-1)/3)+1,
        "day_of_week_num": date.weekday(),
        "day_of_week_eng_abbr": date.strftime('%a'),
        "festival": festival_dict[date.strftime(
            "%Y-%m-%d")] if date.strftime("%Y-%m-%d") in festival_dict else False,
        "is_good_days": True if date in good_days_list else False,
        "is_weekend": True if (date.weekday() == 5) | (date.weekday() == 6) else False,
        "is_public_holiday": True if (date.weekday() == 6) | (date in public_holiday_list) else False,
        "is_wedding_expo": True if date in wedding_expo_list else False
    }
    dim_date_list.append(date_row)
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame(dim_date_list)

In [6]:
from pyspark.sql.types import *
from pyspark.sql import *
# Auxiliar functions
def equivalent_type(f):
    if f == 'datetime64[ns]': return TimestampType()
    elif f == 'int64': return LongType()
    elif f == 'int32': return IntegerType()
    elif f == 'float64': return FloatType()
    else: return StringType()

def define_structure(string, format_type):
    try: typo = equivalent_type(format_type)
    except: typo = StringType()
    return StructField(string, typo)

# Given pandas dataframe, it will return a spark's dataframe.
def pandas_to_spark(pandas_df):
    columns = list(pandas_df.columns)
    types = list(pandas_df.dtypes)
    struct_list = []
    for column, typo in zip(columns, types): 
      struct_list.append(define_structure(column, typo))
    p_schema = StructType(struct_list)
    return spark.createDataFrame(pandas_df, p_schema)

spark_df = pandas_to_spark(df)

In [7]:
RDS_DB="wedding"
RDS_USERNAME="postgres"
RDS_PASSWORD="postgres"
RDS_HOST="final-project-data-warehouse.cit8sojr7959.ap-southeast-1.rds.amazonaws.com"

spark_df.write.format('jdbc')\
    .option("truncate", "true")\
    .option('url', "jdbc:postgresql://{}/{}".format(RDS_HOST, RDS_DB))\
    .option('dbtable', 'dim_date')\
    .option('user', RDS_USERNAME)\
    .option('password', RDS_PASSWORD)\
    .option('driver', 'org.postgresql.Driver')\
    .mode('overwrite')\
    .save()